<a href="https://colab.research.google.com/github/fsilvao/ia_public/blob/main/Algoritmo_genetico_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

# Configuración de parámetros del problema
NUM_CLIENTES = 50  # Número de clientes a los que se realizarán entregas
CAPACIDAD_VEHICULO = 100  # Capacidad máxima de cada vehículo
NUM_VEHICULOS = 5  # Número de vehículos disponibles
TAMANIO_POBLACION = 30  # Tamaño de la población
NUM_GENERACIONES = 100  # Número de generaciones
PROBABILIDAD_CRUZAMIENTO = 0.8  # Probabilidad de cruzamiento
PROBABILIDAD_MUTACION = 0.1  # Probabilidad de mutación

# Generar coordenadas aleatorias para la ubicación de los clientes
def generar_ubicaciones_clientes(num_clientes):
    return [(random.randint(0, 100), random.randint(0, 100)) for _ in range(num_clientes)]

ubicaciones_clientes = generar_ubicaciones_clientes(NUM_CLIENTES)
ubicacion_deposito = (50, 50)  # Ubicación fija del depósito

# Generar demandas aleatorias para cada cliente
def generar_demandas_clientes(num_clientes):
    return [random.randint(1, 20) for _ in range(num_clientes)]

demandas_clientes = generar_demandas_clientes(NUM_CLIENTES)

# Calcular la distancia euclidiana entre dos puntos
def distancia(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

# Calcular el costo total (distancia recorrida) para una ruta
def calcular_costo_ruta(ruta):
    costo = 0
    ubicacion_actual = ubicacion_deposito

    for cliente in ruta:
        costo += distancia(ubicacion_actual, ubicaciones_clientes[cliente])
        ubicacion_actual = ubicaciones_clientes[cliente]

    # Regresar al depósito
    costo += distancia(ubicacion_actual, ubicacion_deposito)
    return costo

# Generar un cromosoma (ruta) inicial
def generar_cromosoma():
    clientes = list(range(NUM_CLIENTES))
    random.shuffle(clientes)
    return clientes

# Crear la población inicial
def crear_poblacion():
    return [generar_cromosoma() for _ in range(TAMANIO_POBLACION)]

# Evaluar el fitness de un cromosoma
def evaluar_fitness(cromosoma):
    costo_total = calcular_costo_ruta(cromosoma)
    return 1 / (costo_total + 1)  # Fitness inverso al costo total

# Selección por ruleta
def seleccion_ruleta(poblacion, fitness_poblacion):
    total_fitness = sum(fitness_poblacion)
    seleccion = random.uniform(0, total_fitness)
    suma = 0
    for i, fitness in enumerate(fitness_poblacion):
        suma += fitness
        if suma >= seleccion:
            return poblacion[i]

# Cruzamiento de un punto
def cruzamiento(padre1, padre2):
    punto_corte = random.randint(1, NUM_CLIENTES - 2)
    hijo1 = padre1[:punto_corte] + [gen for gen in padre2 if gen not in padre1[:punto_corte]]
    hijo2 = padre2[:punto_corte] + [gen for gen in padre1 if gen not in padre2[:punto_corte]]
    return hijo1, hijo2

# Mutación
def mutacion(cromosoma):
    if random.random() < PROBABILIDAD_MUTACION:
        idx1, idx2 = random.sample(range(NUM_CLIENTES), 2)
        cromosoma[idx1], cromosoma[idx2] = cromosoma[idx2], cromosoma[idx1]

# Verificar si la ruta cumple con la restricción de capacidad del vehículo
def cumple_restriccion_capacidad(cromosoma):
    carga_actual = 0
    for cliente in cromosoma:
        carga_actual += demandas_clientes[cliente]
        if carga_actual > CAPACIDAD_VEHICULO:
            return False
    return True

# Función principal del algoritmo genético
def algoritmo_genetico():
    poblacion = crear_poblacion()

    for generacion in range(NUM_GENERACIONES):
        # Evaluar la población
        fitness_poblacion = [evaluar_fitness(cromosoma) for cromosoma in poblacion]

        # Imprimir el mejor resultado de la generación actual
        mejor_fitness = max(fitness_poblacion)
        print(f"Generación {generacion + 1}, Mejor fitness: {mejor_fitness}")

        nueva_poblacion = []

        while len(nueva_poblacion) < TAMANIO_POBLACION:
            # Selección de padres
            padre1 = seleccion_ruleta(poblacion, fitness_poblacion)
            padre2 = seleccion_ruleta(poblacion, fitness_poblacion)

            # Cruzamiento
            if random.random() < PROBABILIDAD_CRUZAMIENTO:
                hijo1, hijo2 = cruzamiento(padre1, padre2)
            else:
                hijo1, hijo2 = padre1, padre2

            # Mutación
            mutacion(hijo1)
            mutacion(hijo2)

            # Verificar restricciones de capacidad
            if cumple_restriccion_capacidad(hijo1):
                nueva_poblacion.append(hijo1)
            if cumple_restriccion_capacidad(hijo2) and len(nueva_poblacion) < TAMANIO_POBLACION:
                nueva_poblacion.append(hijo2)

        poblacion = nueva_poblacion

    # Evaluar la población final y seleccionar el mejor resultado
    fitness_poblacion = [evaluar_fitness(cromosoma) for cromosoma in poblacion]
    mejor_cromosoma = poblacion[fitness_poblacion.index(max(fitness_poblacion))]

    print("\nMejor ruta encontrada:")
    print(mejor_cromosoma)
    print("Fitness:", max(fitness_poblacion))

# Ejecutar el algoritmo genético
algoritmo_genetico()


Generación 1, Mejor fitness: 0.0004367831258819593
